# 精准营销应用

## 6.1 精准营销应用
精准营销建立在客户历史数据的基础之上，通过客户之前的消费习惯来推测之后的消费倾向。通过分析前面构建的客户标签体系来个性化营销方案，这里我们主要参考两种方式：
+ 商品兴趣度排行榜的构建：即计算客户对某类商品的兴趣度，根据兴趣度的排名进行营销。
+ 目标客户的筛选：根据客户标签体系，筛选满足固定特点的群体进行营销。


计算商品兴趣度排行榜时，将时间衰变、tf-idf文本权重和消费金额三个指标进行加权来综合计算总得分。某一类商品的客户兴趣排行榜示意如下：

<center><img src=".\Pics\Pic6_1.jpg" width=600 height = 600 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图6.1：彩票类消费客户兴趣度排行榜</font></center>

对目标客户进行自定义筛选时，根据客户标签体系，筛选满足固定特征的群体，进行精准营销。某一类客户的特征和筛选结果示意如下：

<center><img src=".\Pics\Pic6_2.jpg" width=600 height = 600 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图6.2：目标客户筛选</font></center>

## 6.2 基于时间的商品兴趣度计算方法
时间衰变算法在很多行业都会被应用，就像电商行业，在给客户推荐商品时，会分析客户对于平台商品的兴趣偏好度，同时这个兴趣偏好度也会随着时间的流逝而发生变化。

遗忘曲线是由德国心理学家艾宾浩斯（Hermann Ebbinghaus ）研究发现的，其描述了人类大脑对新事物遗忘的规律，人们可以从遗忘曲线中掌握遗忘规律并加以利用，从而提升自我记忆的能力。人的记忆衰变过程，如下图所示：

<center><img src=".\Pics\Pic6_3.jpg" width="450" height="125" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图6.3：艾宾浩斯遗忘曲线</font></center>

+ 这条曲线告诉人们在学习中的遗忘是有规律的，遗忘的进程很快，并且先快后慢。在分析客户对电商平台的商品兴趣偏好时，也可以借鉴遗忘曲线。
+ 对于电商平台来说，通过营销活动达到的目的是关心客户的“短期兴趣”。看看客户短期内会更倾向于购买哪些宝贝，从而更好地去做精准营销，如短信、站内广告等。

如果客户在观察点之前多个时刻($t_0，t_1，t_2 ...$)购买某类商品，则每个时刻与观察点的距离为$\Delta t_0，\Delta t_1，\Delta t_2$，对于该商品的兴趣度衰变曲线如下图所示：

<center><img src=".\Pics\Pic6_4.jpg" width="410" height="125" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图6.4：客户兴趣度衰变曲线</font></center>


每个时刻兴趣度$S_t$的计算公式如下：
$$S_t = e^{-\eta \Delta t}$$
+ $\Delta t$表示消费时刻$t$距离观察点的月数
+ $\eta$ 表示衰减因子

考虑到客户会出现多次消费行为(这里默认每次消费行为的衰减因子一致)，对某商品的兴趣总和公式如下：
$$ S = \sum e^{-\eta \Delta t} $$

假定此处为彩票类商品消费，我们设定衰减因子 $\eta$ 为0.5，我们可以快速计算出每个时刻的兴趣度，将每个时刻的兴趣度相加，得到总的基于时间的商品兴趣度。

除了考虑基于时间的兴趣度，还要考虑基于tf-idf和消费金额的兴趣度，做一个综合的评估。

## 6.3 时间差的计算
前面已经了解了时间衰减机制以及具体的商品兴趣度计算方法，基于时间的商品兴趣度计算和RFM模型中的近度(Recency)类似，最关键的是找到一个标准的观察点，计算每笔消费距离观察点的时间差。

在前面的步骤中，我们依据交易频度随时间的变化，限定了观察的有效时间，即`2016-07-01`到`2017-12-31`。所以在兴趣度的计算中，我们选取的观察点也为`2018-01-01`，由此就可以根据每笔消费的时间计算距离月数。

商品兴趣度计算的前提是提取只属于消费类的数据，在5.3步骤中已计算过，计算步骤为从`data`中选取金额流出的交易记录，然后再剔除转账、体现、转入、还款等记录，我们在这里回顾一下`consume`表。

<center><font size="2.5px" face="微软雅黑" color="666666">表6.1：consume表内容示例</font></center>
<center><img src=".\Pics\Table6_1.jpg" alt="cursor" align=center /></center>

> 实训任务
> + `consume`表在题目中已预先读入，保存在DataFrame对象 `consume`中。
> + 请确立观察点为`2018-01-01`，保存在变量`standard_day`中。
>   + 提示：Pandas中的`Timestamp`可以设定时间戳作为我们的观察点。
> + 请新建一列`distance`，计算`consume`中每次消费距离观察点的月数，保存在`consume['distance']`中。
>   + 提示：Pandas中的`Timedelta`表示两个`datetime64`对象之间的时间差，如`pd.Timedelta(days=1)`代表一天的时间差，但是`Timedelta()`的参数中没有月的单位，这里我们使用`days=30`代表一月。

可以看到，我们已经计算好了每笔消费距离观察点的月数，保存在了`consume['distance']`中

In [ ]:
import pandas as pd

# 选取观察点
standard_day = pd.Timestamp(2018, 1, 1)

# 计算月数
consume['distance'] = (standard_day - consume['pay_time']) / pd.Timedelta(days=30)

print(consume.head(2))

## 6.4 基于时间的商品兴趣度计算
前面我们已经计算了每笔消费距离观察点的月数，现在我们根据时间衰减算法计算基于时间的商品兴趣度。在这里我们以彩票类消费为例，其他类型的商品分析流程类似。

首先提取出彩票类消费的交易记录，使用文本匹配的方法，将文本类标签提取出的彩票类消费的关键词作为匹配词，即`text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'`，匹配出相对应的交易记录。接着按每个客户进行分组，以此来计算每个客户对彩票类商品基于时间的商品兴趣度。

> 实训任务
> + 请根据`text_list`，使用词匹配的方法找出`consume`的交易附言列(`describe`)中彩票类消费的记录，保存在DataFrame对象`lottery_ticket`中，所有消费数据`consume`已预先读入，此步骤题目已默认给出。
> + 请根据6.2中的公式，计算每个客户基于时间的商品兴趣度。对消费数据按客户进行分组，对时间差按照时间衰减公式进行聚合运算，将结果保存在`user_features`表的`time_penalty`列中。
> + 衰减因子$\eta$(设定为0.5)和彩票类的文本标签已经给出，变量分别为`eta`和`text_list`。

In [ ]:
import numpy as np
import pandas as pd

eta = 0.5
text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 筛选彩票类的消费记录
lottery_ticket = consume[consume['describe'].str.contains(text_list)]

# 基于时间的商品兴趣度
user_features['time_penalty'] = lottery_ticket.groupby('user_id')['distance'].apply(lambda x : sum(np.exp(-eta*x)))

print(user_features['time_penalty'].head())

## 6.5 基于消费金额的商品兴趣度计算
前面我们已经计算了基于时间的商品兴趣度，接下来我们计算基于消费金额的商品兴趣度。

计算方法和基于时间的兴趣度类似，首先提取出彩票类消费的交易记录，使用文本匹配的方法，匹配出相对应的交易记录。接着按每个客户进行分组，将每个客户的消费金额进行加和，以此来计算每个客户对彩票类商品基于消费金额的商品兴趣度。

> 实训任务
> + 请根据`text_list`，使用词匹配的方法找出`consume`中彩票类消费的记录，保存在DataFrame对象`lottery_ticket`中，所有消费数据`consume`已预先读入，此步骤已默认给出。
> + 请使用`groupby()`函数对`lottery_ticket`根据`user_id`列进行分组，计算每个客户对彩票类商品的基于消费金额的商品兴趣度，保存在`user_features`表的`payment_sum`列中。

In [ ]:
import numpy as np
import pandas as pd

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 筛选彩票类的消费记录
lottery_ticket = consume[consume['describe'].str.contains(text_list)]

# 计算消费金额
user_features['payment_sum'] = lottery_ticket.groupby('user_id')['payment'].sum()

print(user_features['payment_sum'].head())

## 6.6 基于tf-idf的商品兴趣度计算
我们已经计算了基于时间和消费金额的商品兴趣度，接下来我们计算基于`tf-idf`的商品兴趣度，计算方法为将客户对某一类商品文本标签的`tf-idf`权重值求和。

在前面我们已经使用TfidfVectorizer模型提取了文本标签的`tf-idf`值，保存在了`user_features`表中，我们首先需要筛选出存放彩票类消费的文本标签`tf-idf`权重值的列。

> 实训任务
> + 请从`user_features`表中选取彩票类消费文本标签的`tf-idf`值，保存在变量`lottery_tfidf`中，此步骤已默认给出。
> + 请计算每个客户对彩票类商品的基于`tf-idf`的商品兴趣度(将`lottery_tfidf`按行求和)，保存在`user_features`表的`tfidf_sum`列中。

In [ ]:
import pandas as pd

# 选取字段
lottery_tfidf = user_features[['停彩','大乐透','双色球','福利彩票','彩票','竞彩','追号']]

# tfidf求和
user_features['tfidf_sum'] = lottery_tfidf.sum(axis = 1)

print(user_features['tfidf_sum'].head())

## 6.7 数据归一化
在分别计算了客户彩票类消费的三个兴趣度之后，我们要计算三个指标的加和来综合计算客户的商品兴趣度。三个指标的量纲不同，如消费金额兴趣度的数值要远远大于`tfidf`和时间上的兴趣度，所以我们需要进行归一化，将三个指标的数据都缩放在`(0,1)`区间，统一它们的量纲。

在前面我们已经计算并将三个兴趣度融合在了`user_features`表中，列名分别为`time_penalty、tfidf_sum、payment_sum`。

> 实训任务
> + 请使用`apply()`函数对`user_features`表中的基于时间的兴趣度进行`sigmoid`归一化，并保存在原列中。
> + 请使用`apply()`函数对`user_features`表中的基于消费金额和tf-idf权重的兴趣度进行`min-max`归一化，并保存在原列中。
> + 提示：
>   + `sigmoid`归一化的计算方法为： $s(x) = \frac{1}{1+e^{-x}}$  
>   + `min-max`归一化的计算方法为： $s(x) = \frac{x-x.min()}{x.max()-x.min()}$  

In [ ]:
import pandas as pd
import numpy as np

# sigmoid归一化
user_features['time_penalty'] = user_features[['time_penalty']].apply(lambda x:1/(1+np.exp(-x)))

# min-max归一化
user_features['payment_sum'] = user_features[['payment_sum']].apply(lambda x : (x-x.min()) / (x.max() - x.min()))
user_features['tfidf_sum'] = user_features[['tfidf_sum']].apply(lambda x : (x-x.min()) / (x.max() - x.min()))

print(user_features[['time_penalty','payment_sum','tfidf_sum']].head(10))

## 6.8 商品兴趣度排行榜的综合计算
前面我们已经计算了基于时间、消费金额和`tf-idf`的商品兴趣度，三项指标均可以作为衡量客户在这一商品类别的兴趣度评价指标。但是单一用一个指标无法综合的衡量客户的消费情况，如有些客户消费金额低，但是消费频率高(`tf-idf`高)，时间距离近(时间衰减弱)，这些客户仍然具有很高的推销价值，所以我们将三个归一化后的指标进行相加，计算`final_score`以便于更合理的综合评估客户在这一商品类别的兴趣度。

> 实训任务
> + 请将三个商品兴趣度相加计算`final_score`，保存在`user_features`表的`final_score`列中。
> + 请将`user_features`表根据`final_score`进行降序排序，取前`10`的客户，保存在变量`top10`中。

可以看到排名第一的客户，消费金额兴趣度为1，代表在彩票类消费金额上此客户最多。同时该客户的时间兴趣度上也相对较高，可以初步验证我们排名的正确性。接下来再以其他的方式进一步验证排名的准确性。

In [ ]:
import pandas as pd

# 计算final_score
user_features['final_score'] = user_features['time_penalty'] + user_features['payment_sum'] + user_features['tfidf_sum']

# top10
top10 = user_features.sort_values('final_score', ascending=False)[:10]

print(top10[['time_penalty','payment_sum','tfidf_sum','final_score']])

## 6.9 交易次数的可视化
前一步中，我们已经计算出了客户彩票类消费的排行榜`top10`，现在我们通过画出前三位客户的彩票类交易次数折线图，以此来验证我们排行榜的正确性。

> 实训任务
> + `top10`已预先读入，请从`top10`中选取排名前3的`user_id`，保存在变量`top3`中，此步骤已默认给出。
> + 属于消费的数据`consume`已预先读入，文本关键词`text_list`已给出。
> + 请根据选出的`user_id`筛选出三个客户各自的交易记录，通过匹配文本关键词的方法，选出三个客户分别属于彩票类交易的消费记录，此步骤在题目中已默认给出。
> + 请按天分组，分别计算排名前3的客户的彩票类交易次数，并使用Pandas中的`plot()`函数绘制折线图（一个图中三条线），线的类型在`styles`列表中已给出。
> + 请设置图的标题为`彩票交易次数变化`。
> + 请设置`x`,`y`轴标签分别为`日期`和`交易次数`。
> + 请设置图例名称分别为`'第1名', '第2名','第3名'`。

从图中可以看出：
+ 排名第一的客户彩票类消费时间主要集中在2016年9月-2017年9月，之后的消费出现停滞；
+ 排名第二的客户彩票类消费距离观察点很近；
+ 排名第三的客户在2017年5月出现一个交易次数的峰值。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 选择排名前3的user_id
top3 = top10.index.values[:3]

# 不同ID的客户用不同的线段表示
styles=['--','-',":"]

fig = plt.figure(figsize=(15,6))

# 绘制折线图
for user_id,style in zip(top3,styles):
     # 筛选用户的交易记录
    top_select = consume.loc[consume['user_id'] == user_id]
     # 匹配文本关键词
    top_select = top_select[top_select['describe'].str.contains(text_list)]
     # 按天分组，计算交易次数并绘图
    top_select.groupby(top_select['pay_time'].dt.date).size().plot(style=style)

# 设置标题
plt.title("彩票交易次数变化")

# 设置x,y轴标签
plt.xlabel("日期")
plt.ylabel("交易次数")

# 设置图例
plt.legend(["第1名", "第2名", "第3名"])

# 设置日期显示格式
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.show()

## 6.10 交易金额的可视化
上一步我们从交易次数上绘制了排名前三的客户的彩票类交易次数随时间的交易曲线，现在我们再从交易金额上绘制这三名客户彩票类交易金额随时间的交易曲线。

> 实训任务
> + `top10`题目已预先读入，选取排名前3的`user_id`，保存在变量`top3`中。
> + 属于消费的数据`consume`已预先读入，文本关键词`text_list`已给出。
> + 请根据选出的`user_id`筛选出三个客户各自的交易记录，通过匹配文本关键词的方法，选出三个客户分别属于彩票类交易的消费记录，此步骤题目中已默认给出。
> + 请按天分组，分别计算排名前3的客户的彩票类交易金额，并使用Pandas中的`plot()`函数绘制折线图（一个图中三条线），线的类型在`styles`列表中已给出。
> + 请绘制排名前3客户的彩票交易金额折线图。
> + 请设置图的标题为`彩票交易金额变化`。
> + 请设置x,y轴标签分别为`日期`和`消费金额`。
> + 请设置图例名称分别为`'第1名', '第2名','第3名'`。

从图中可以看出：

+ 第一名在时间维度和消费次数上都不占优势，但是明显可以看出该客户的消费金额远远高于第二和第三名。在2016年10月的一次彩票类消费超过了10000元，可以看出这是一名狂热的彩票爱好者。
+ 第二名彩票类的消费次数远少于第三名，且消费金额基本持平，但在时间维度上占据很大的优势，综合得分上超过第三名。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

text_list = '停彩|大乐透|双色球|福利彩票|彩票|竞彩|追号'

# 选择排名前3的user_id
top3 = top10.index.values[:3]

# 不同ID的客户用不同的线段表示
styles = ['--','-',":"]

fig = plt.figure(figsize=(15,6))

for user_id,style in zip(top3,styles):
    # 筛选用户的交易记录
    top_select = consume.loc[consume['user_id'] == user_id]
    # 匹配文本关键词
    top_select = top_select[top_select['describe'].str.contains(text_list)]
    # 按天分组，计算交易金额并绘图
    top_select.groupby(top_select['pay_time'].dt.date)['payment'].sum().plot(style = style)

# 设置标题
plt.title("彩票交易金额变化")

# 设置x,y轴标签
plt.xlabel("日期")
plt.ylabel("消费金额")

# 设置图例
plt.legend(["第1名", "第2名", "第3名"])

# 设置日期显示格式
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.show()

## 6.11 目标客户的筛选
寻找目标客户进行广告投放是精准营销中的重要一环，通常来说，寻找的目标客户要有对产品与服务的需求。通过丰富的人群定向推送广告，实现精准投放。

在前面的分析中，我们以彩票类商品消费为例，计算出了客户兴趣度的排行榜，同理，我们可以根据文本标签划分出的几个类别对其他商品进行兴趣度排行的计算。在推销某类商品时，我们可以参考排行榜，针对性的选取排名靠前的部分客户，这是寻找目标客户的一种方式。

另外一种方式为营销人员根据实际的业务需求，针对性的选取几个客户的关键特点，从数据库中精准查询到符合需求的目标客户。如我们要推销某出行app联名的分期信用卡，为200个目标客户推送广告，根据人为经验锁定目标的特征为：
+ 非休眠客户
+ 有商旅消费的记录
+ 有信用卡的使用记录
+ 有消费分期的需求(使用记录)

我们已经事先把`user_features`表存储在数据库中，存放在了`transactions`库中的`user_features`表中，此时`user_id`为表中的第一个字段。

> 实训任务
> + 请构建连接对象`connection`和游标对象`cursor`，并补全sql语句，要求从数据表`user_features`中查询出`business_travel_cnt > 0`、`credit_card_repay_cnt > 0`、`is_installment = 1`、`sleep_customers = 0`的所有客户信息，并按照`credit_card_repay_cnt`降序排序，取前200个客户，命名为`collected_users`。
> + 将`collected_users`转换为DataFrame格式，并打印`collected_users`中的商旅消费次数和信用卡还款次数，此部分题目已默认完成。

下面的表格中提供了连接对象`connection`中所需要的参数信息：

<center><font size="2.5px" face="微软雅黑" color="666666">表6.3：MySQL连接参数表</font></center>

|参数名称|参数信息|参数类型|
|:----|:-----|:----|
|host|traindb.cookdata.cn|string|
|port|3306|int|
|user|raa_user|string|
|password|bigdata123|string|
|db|transactions|string|
|charset|utf8mb4|string|

可以看到，通过条件的筛选，我们定位了200名目标客户。此部分客户的商旅消费次数和信用卡还款次数较高，营销人员可以进一步的缩小选取范围，这对于有针对性地推广意义重大。

In [ ]:
import pymysql
import pandas as pd

# 构建连接对象
connection = pymysql.connect(host='traindb.cookdata.cn', 
                             port=3306, 
                             user='raa_user', 
                             password='bigdata123', 
                             db='transactions',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM user_features \
        WHERE business_travel_cnt > 0 AND credit_card_repay_cnt > 0 \
        AND is_installment = 1 AND sleep_customers = 0 \
        ORDER BY credit_card_repay_cnt DESC LIMIT 200"
        cursor.execute(sql)
        collected_users = cursor.fetchall()

finally:
    connection.close()

# 查看筛选出的客户信息
collected_users = pd.DataFrame(collected_users)
print(collected_users[['user_id','business_travel_cnt','credit_card_repay_cnt']])